# Вычислительная математика. Домашняя лабораторная работа №2.
### Выполнил: Хайдари Фарид, группа Б01-909.

## Подготовка среды

In [173]:
# Включаем все необходимые библиотеки
import numpy as np

# Задаем глобальные константы
SIZE = 49
EPS = 1e-12

## Функция для заполнения матрицы

In [164]:
def initA(n: int, calc_a, calc_b, calc_c, calc_p):
  A = np.zeros((n + 1, n + 1))

  # Прописываем первую строчку
  A[0][0] = calc_b(0)
  A[0][1] = calc_c(0)

  # Идем по диагонали
  for i in range(1, n):
    A[i][i-1] = calc_a(i)
    A[i][i]   = calc_b(i)
    A[i][i+1] = calc_c(i)

  # Идем по последней строчке:
  for i in range(n + 1):
    A[-1][i] = calc_p(i)

  return A


## Функции для вычисления нормы матрицы и вектора

In [165]:
def calcNorm1(mx):
  maxSum = 0
  rowNum, colNum = mx.shape
  for j in range(colNum):
    sum = 0
    for i in range(rowNum):
      sum += abs(mx[i][j])
    maxSum = max(sum, maxSum)
  return maxSum

def calcNorm2(mx):
  rowNum, colNum = mx.shape
  maxSum = 0
  for i in range(rowNum):
    maxSum = max(sum(abs(mx[i])), maxSum)
  return maxSum

def calcNorm3(mx):
  return np.sqrt(max(abs(np.linalg.eigvals(np.dot(mx, mx.transpose())))))


def calcVecNorm1(vec):
  return max(abs(vec))[0]

def calcVecNorm2(vec):
  return sum(abs(vec))[0]

def calcVecNorm3(vec):
  return np.sqrt(sum(abs(vec)**2))[0]

## Определяем методы для прямого метода Гаусса

In [166]:
def mainElemIdx(mx, idx):
  main_elem = mx[idx][idx]
  main_idx = idx

  for i in range(idx, len(mx)):
    if (mx[i][idx] > main_elem):
      main_elem = mx[i][idx]
      main_idx = i

  return main_idx

def forwardGauss(mx):
  rowNum = len(mx)
  iterEnd = min(rowNum, len(mx[0]))

  for i in range(iterEnd):
    mainIdx = mainElemIdx(mx, i) # Найти главный элемент
    mx[[mainIdx, i]] = mx[[mainIdx, i]] # Поменять строки матрицы
    for j in range(i + 1, rowNum):
      mx[j] -= mx[j][i] / mx[i][i] * mx[i] # Вычесть строку с главным элементом

  return mx

def backwardGauss(mx):
  rowNum, colNum = mx.shape
  assert rowNum <= colNum

  ans = np.zeros((colNum - 1, 1))
  for i in range(rowNum - 1, -1, -1):
    sum = 0
    for j in range(i + 1, colNum - 1):
      sum += mx[i][j] * ans[j][0]
    ans[i][0] = (mx[i][rowNum] - sum) / mx[i][i]

  return ans


## Определяем функции для итерационного метода Зейделя

In [167]:
def getDLU(mx):
  rowNum, colNum = mx.shape
  # assert rowNum == colNum
  sz = rowNum

  D = np.zeros((sz, sz))
  U = np.zeros((sz, sz))
  L = np.zeros((sz, sz))

  for i in range(sz):
    for j in range(sz):
      if i == j:
        D[i][j] = A[i][j]
      elif i > j:
        L[i][j] = A[i][j]
      else:
        U[i][j] = A[i][j]
  
  return D, L, U

def runZeidel(mx, accuracy, calcNorm):
  done = False
  iterNum = 0
  x = np.zeros((len(mx), 1))

  D, L, U = getDLU(mx)
  LpDinv = np.linalg.inv(L + D)
  LpDinvU = np.dot(LpDinv, U)

  while not done:
    iterNum += 1
    newX = -np.dot(LpDinvU, x) + np.dot(LpDinv, f)
    done = calcNorm(newX - x) < accuracy
    x = newX

  return x, iterNum


## Инициализируем матрицу системы, вектор правых частей и расширенную матрицу системы

In [169]:
ret_one = lambda i: 1
A = initA(SIZE, ret_one, lambda i: 5, ret_one, ret_one) # Матрица системы
f = np.transpose([np.arange(1, SIZE + 2)]) # Вектор правых частей

expandedA = np.concatenate((A, f), axis=1) # Получаем расширенную матрицу системы

print(A)
print('')
print(np.transpose(f))
print('')
print(expandedA)

[[5. 1. 0. ... 0. 0. 0.]
 [1. 5. 1. ... 0. 0. 0.]
 [0. 1. 5. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 1. 0.]
 [0. 0. 0. ... 1. 5. 1.]
 [1. 1. 1. ... 1. 1. 1.]]

[[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
  25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
  49 50]]

[[ 5.  1.  0. ...  0.  0.  1.]
 [ 1.  5.  1. ...  0.  0.  2.]
 [ 0.  1.  5. ...  0.  0.  3.]
 ...
 [ 0.  0.  0. ...  1.  0. 48.]
 [ 0.  0.  0. ...  5.  1. 49.]
 [ 1.  1.  1. ...  1.  1. 50.]]


## Получим число обусловленности для матрицы системы

In [170]:
AInv = np.linalg.inv(A)
calcNorms = [calcNorm1, calcNorm2, calcNorm3]

for idx, calcNorm in enumerate(calcNorms):
  print(f'Число обусловленности матрицы A по норме {idx + 1}: {calcNorm(A) * calcNorm(AInv)}')

Число обусловленности матрицы A по норме 1: 12.220201853215574
Число обусловленности матрицы A по норме 2: 486.4664631877187
Число обусловленности матрицы A по норме 3: 17.464946353215716


## Получаем невязку для метода Гаусса

In [171]:
ug = backwardGauss(forwardGauss(expandedA)) # Получаем ответ
diffGauss = f - np.matmul(A, ug) # Вектор с ошибками

normVals = [
  calcVecNorm1(diffGauss), # Вычисляем невязку по второй норме
  calcVecNorm2(diffGauss), # Вычисляем невязку по второй норме
  calcVecNorm3(diffGauss)  # Вычисляем невязку по второй норме
]

for idx, normVal in enumerate(normVals):
  print(f'Невязка метода Гаусса для нормы {idx + 1}: {normVal}')


Невязка метода Гаусса для нормы 1: 7.105427357601002e-15
Невязка метода Гаусса для нормы 2: 5.3290705182007514e-14
Невязка метода Гаусса для нормы 3: 1.6604423698964792e-14


## Получаем невязку для метода Зейделя

In [174]:
vecNorms = [
  calcNorm1,
  calcNorm2,
  calcNorm3
]

for idx, calcVecNorm in enumerate(vecNorms):
  uz, iterNum = runZeidel(expandedA, EPS, calcVecNorm)
  diffZeidel = f - np.matmul(A, uz)
  print(f'Невязка метода Зейделя для нормы {idx + 1}: {calcNorm(diffZeidel)}, количество итераций: {iterNum}')

Невязка метода Зейделя для нормы 1: 1.5407591776796986e-13, количество итераций: 25
Невязка метода Зейделя для нормы 2: 5.494410184266636e-13, количество итераций: 24
Невязка метода Зейделя для нормы 3: 5.494410184266636e-13, количество итераций: 24
